In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

In [ ]:
## Data Preparation
dat = pd.read_csv("data/train.csv")
## With and without the -1, because it could be of significance.
count_nan = len(dat) - dat.count()
print("Number of NaNs", count_nan)
X = dat.loc[:, dat.columns != "target"]
y = dat.loc[:, "target"]

print("Values of target", y.unique())
print("Number of 1:", sum(y == 1), "Percentage of dataset:", sum(y==1)/len(y)*100, "%")
print("Number of 0:", sum(y == 0), "Percentage of dataset:", sum(y==0)/len(y)*100, "%")
X_no_neg = X
y_no_neg = y
cat = pd.DataFrame()
bn = pd.DataFrame()
norm = pd.DataFrame()
ids = X['id']

## We ignore the id column for this step
for i in X.columns[1:]:
    if "cat" in i:
        # Transform Median
        cat[i] = dat.loc[:, i]
    elif "bin" in i:
        # Transform
        bn[i] = dat.loc[:, i]
    else:
        # Do something
        norm[i] = dat.loc[:, i]
for i in cat.columns:
    tmp = 0
    cat.loc[cat[i] == -1, i] = np.median(cat.loc[cat[i] != -1,i])
    tmp = tmp + sum(cat[i] == -1)
    
print("Number of -1 in data:", tmp)

for i in bn.columns:
    tmp = 0
    tmp = tmp + sum(bn[i] == -1)
    print(i, ":", tmp)
    
print("There are no negatives in bin columns")
for feature in cat:
    temp.append(pd.get_dummies(cat[feature]))

new_cat = pd.DataFrame()
for feature in temp:
    new_cat = pd.concat([new_cat, feature], axis=1)
pipe = Pipeline([
       ("remove_neg_ones", Imputer(missing_values=-1, strategy="mean")),
        ("z-scaling", StandardScaler())
        ])

scaled_norm = pd.DataFrame(pipe.fit_transform(norm), columns = norm.columns)
X_final = pd.concat([bn, new_cat, scaled_norm], axis=1)
clf = DecisionTreeRegressor()
clf.fit(X_final, y)
features = clf.feature_importances_
important_features = X_final[X_final.columns[np.argsort(-features)[:10]]]

def gini(actual, pred, cmpcol = 0, sortcol = 1):
     assert( len(actual) == len(pred) )
     all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
     all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
     totalLosses = all[:,0].sum()
     giniSum = all[:,0].cumsum().sum() / totalLosses
 
     giniSum -= (len(actual) + 1) / 2
     return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

from sklearn.ensemble import GradientBoostingClassifier

X = pd.concat([bn, new_cat, scaled_norm], axis=1)

scale = StandardScaler()
scaled_X = scale.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y)

clf = GradientBoostingClassifier(n_estimators=50, max_depth=3)
clf.fit(X_train, y_train)

predicted = clf.predict(X_test)
print("Confusion matrix")
print(confusion_matrix(y_test, predicted))
print("Accuracy:", accuracy_score(y_test, predicted)*100, "%")
X_ones = scaled_X[y==1, :]
y_ones = y[y==1]
X_zeroes = scaled_X[y==0, :]
dat_ones = pd.DataFrame(X_ones)
dat_ones["target"] = [1] * len(dat_ones)
dat_f = pd.DataFrame(X_zeroes)
dat_f["target"] = np.zeros(len(dat_f))
dat_ones = dat_ones.append(dat_f.sample(n=(len(dat_ones))))
print("Number of 1 after rebuilding data:", sum(dat_ones.target == 1), "Percentage of dataset:", sum(dat_ones.target==1)/len(dat_ones.target)*100, "%")
print("Number of 0 after rebuilding data:", sum(dat_ones.target == 0), "Percentage of dataset:", sum(dat_ones.target==0)/len(dat_ones.target)*100, "%")

print("Reclassifing")
dat_ones.target
X_shrunk = dat_ones.loc[:, dat_ones.columns != "target"]
y_shrunk = dat_ones.target
scale = StandardScaler()
scaled_X = scale.fit_transform(X_shrunk)

X_train, X_test, y_train, y_test = train_test_split(X_shrunk, y_shrunk)

clf = GradientBoostingClassifier(n_estimators=50, max_depth=3)
clf.fit(X_train, y_train)

predicted = clf.predict(X_test)
print("Confusion matrix")
print(confusion_matrix(y_test, predicted))
print("Accuracy of new set:", accuracy_score(y_test, predicted)*100, "%")
print("Gini for Gradient Boosting:", gini_normalized(y_test, predicted))

from sklearn.ensemble import RandomForestClassifier
print("Predicting from RandomForestClassifier")
rfc = RandomForestClassifier(n_estimators = 500)
rfc.fit(X_train, y_train)
predicted_rfc = rfc.predict(X_test)
print(confusion_matrix(y_test, predicted_rfc))
print("Accuracy of new set as RFC:", accuracy_score(y_test, predicted_rfc)*100, "%")
print("Gini for Random Forest:", gini_normalized(y_test, predicted_rfc))
X_train, X_test, y_train, y_test = train_test_split(X_shrunk, y_shrunk, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

gnb = GaussianNB()
dtc = tree.DecisionTreeClassifier(max_depth=3)
rndf = RandomForestClassifier()

gnb.fit(X_train, y_train)
gnbpred = gnb.predict(X_val)
print ("GNB Classifier Accuracy", accuracy_score(y_val, gnbpred)*100, "%")
print("Gini for GNB:", gini_normalized(y_val, gnbpred))
dtc = dtc.fit(X_train, y_train)
dtcpred = dtc.predict(X_val)
print ("Decision Tree Classifier Accuracy", accuracy_score(y_val, dtcpred)*100, "%")
print("Gini for Decision Tree:", gini_normalized(y_val, dtcpred))
rndf.fit(X_train, y_train)
rndfpred = rndf.predict(X_val)
#print ("Random Forest Accuracy", accuracy_score(y_val, rndfpred)*100, "%")
#print("Gini for Random Forest:", gini_normalized(y_val, rndfpred))


X_b = np.empty([len(y_val), 3])
X_b[:, 0] = gnbpred
X_b[:, 1] = dtcpred
X_b[:, 2] = rndfpred
y_b = y_val
rfcA = RandomForestClassifier()
rfcA.fit(X_b, y_b)

pred = np.empty([len(y_test), 3])
l = [gnb,dtc,rndf]
tmp = 0
for i in l:
    pred[:,tmp] = i.predict(X_test)
    tmp = tmp+1
predictions = rfcA.predict(pred)
print ("MYvoting classifier Accuracy", accuracy_score(y_test, predictions)*100, "%")
print("Gini for MYvoting classifier:", gini_normalized(y_test, predictions))

vote = VotingClassifier(estimators=[('gnb', gnb),('rf', rndf), ('dtc', dtc)], voting='hard')
vote.fit(X_train, y_train)
votepred = vote.predict(X_test)
print ("\nVoting Classifier", accuracy_score(y_test, votepred)*100, "%")
print("Gini for Voting Classifier:", gini_normalized(y_test, votepred))

class StackingRegressor:
    def __init__(self, l_estimators):
        self.l_estimators = l_estimators
        self.models = []
        self.blender = None
        
    def fit(self, X, y):
        # Make a matrix of predictions
        X_train, X_test, y_train, y_test = train_test_split(X, y)
        for i in self.l_estimators:
            #bag_clf = BaggingRegressor(LogisticRegression(), n_estimators=50, max_samples=int(len(y)*.75), bootstrap=True, n_jobs=-1)
            #bag_clf.fit(X, y)
            model = i.fit(X_train,y_train)
            self.models.append(model)
        predictions = np.empty([len(y_test), len(self.l_estimators) + 1])
        tmp = 0
        for i in self.models:
            predictions[:,tmp] = i.predict(X_test)
            tmp = tmp + 1
        predictions[:,len(self.l_estimators)] = y_test
        rfcA = RandomForestClassifier()
        Y = y_test
        x = predictions[:,0:tmp]
        rfcA.fit(x, Y)
        self.blender = rfcA
        
    def predict(self, X):
        predictions = np.empty([len(X), len(self.l_estimators)])
        tmp = 0
        for i in self.models:
            predictions[:,tmp] = i.predict(X)
            tmp = tmp + 1
        return self.blender.predict(predictions)
    
    def score(self, X, y):
        s = StackingRegressor(l_estimators = self.l_estimators)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
        s.fit(X_train,y_train)
        pred = s.predict(X_test)
        return accuracy_score(y_test, pred)
    
X_train, X_test, y_train, y_test = train_test_split(X_shrunk, y_shrunk)
gnb = GaussianNB()
dtc = tree.DecisionTreeClassifier(max_depth=3)
rndf = RandomForestClassifier()
stacker = StackingRegressor(l_estimators=[gnb,dtc,rndf])
stacker.fit(X_train, y_train)
predict = stacker.predict(X_test)

print ("Stacking Accuracy:", accuracy_score(y_test, predict) * 100, "%")

print("Gini for blender:", gini_normalized(y_test, predict))

X_train, X_test, y_train, y_test = train_test_split(X_final, y)

clf = GradientBoostingRegressor()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
confusion_matrix(y_test, np.round(predicted))

print(f"Gini score for default GradientBoostingRegressor {gini_normalized(y_test, predicted)}")

X_train, X_test, y_train, y_test = train_test_split(X_shrunk, y_shrunk)

clf = GradientBoostingRegressor()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
confusion_matrix(y_test, np.round(predicted))

print(f"Gini score for default GradientBoostingRegressor using shrunk dataset {gini_normalized(y_test, predicted)}")

Number of NaNs id                0
target            0
ps_ind_01         0
ps_ind_02_cat     0
ps_ind_03         0
ps_ind_04_cat     0
ps_ind_05_cat     0
ps_ind_06_bin     0
ps_ind_07_bin     0
ps_ind_08_bin     0
ps_ind_09_bin     0
ps_ind_10_bin     0
ps_ind_11_bin     0
ps_ind_12_bin     0
ps_ind_13_bin     0
ps_ind_14         0
ps_ind_15         0
ps_ind_16_bin     0
ps_ind_17_bin     0
ps_ind_18_bin     0
ps_reg_01         0
ps_reg_02         0
ps_reg_03         0
ps_car_01_cat     0
ps_car_02_cat     0
ps_car_03_cat     0
ps_car_04_cat     0
ps_car_05_cat     0
ps_car_06_cat     0
ps_car_07_cat     0
ps_car_08_cat     0
ps_car_09_cat     0
ps_car_10_cat     0
ps_car_11_cat     0
ps_car_11         0
ps_car_12         0
ps_car_13         0
ps_car_14         0
ps_car_15         0
ps_calc_01        0
ps_calc_02        0
ps_calc_03        0
ps_calc_04        0
ps_calc_05        0
ps_calc_06        0
ps_calc_07        0
ps_calc_08        0
ps_calc_09        0
ps_calc_10        0
ps_ca